In [2]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [1]:
!pip install opencv-python numpy scikit-learn xgboost


In [3]:
image_folder = "/content/drive/MyDrive/HAK/candle/Data/Images"


In [5]:
import os

drive_path = "/content/drive/MyDrive/HAK/candle/Data/Images"
if os.path.exists(drive_path):
    print("✅ Dossier trouvé ! Contenu :", os.listdir(drive_path)[:10])
else:
    print("❌ Dossier introuvable ! Vérifie le chemin.")


✅ Dossier trouvé ! Contenu : ['Anomaly', 'Normal']


In [6]:
import cv2
import os

# 📂 Dossier des images originales
input_folder = "/content/drive/MyDrive/HAK/candle/Data/Images"  # 🔥 Modifie selon ton dossier

# 📂 Dossier où enregistrer les images redimensionnées
output_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized"
os.makedirs(output_folder, exist_ok=True)

# 📏 Tailles à tester
resize_sizes = [256, 128, 64]

# 🔄 Redimensionner et sauvegarder
for size in resize_sizes:
    size_folder = os.path.join(output_folder, f"resized_{size}")
    os.makedirs(size_folder, exist_ok=True)

    for category in ["Anomaly", "Normal"]:  # 🔥 Change si tu as d'autres classes
        category_path = os.path.join(input_folder, category)
        size_category_path = os.path.join(size_folder, category)
        os.makedirs(size_category_path, exist_ok=True)

        for img_name in os.listdir(category_path):
            img_path = os.path.join(category_path, img_name)
            img = cv2.imread(img_path)

            if img is not None:
                img_resized = cv2.resize(img, (size, size), interpolation=cv2.INTER_AREA)
                cv2.imwrite(os.path.join(size_category_path, img_name), img_resized)

print("✅ Images redimensionnées et sauvegardées dans :", output_folder)


✅ Images redimensionnées et sauvegardées dans : /content/drive/MyDrive/HAK/candle/Data/Images_Resized


In [7]:
import os
output_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized"

# Vérifier les dossiers créés
print("Dossiers créés :", os.listdir(output_folder))

# Vérifier quelques fichiers redimensionnés
resized_256_path = os.path.join(output_folder, "resized_256")
if os.path.exists(resized_256_path):
    print("Exemples d'images 256x256 :", os.listdir(resized_256_path)[:10])
else:
    print("❌ Le dossier resized_256 n'a pas été créé !")


Dossiers créés : ['resized_256', 'resized_128', 'resized_64']
Exemples d'images 256x256 : ['Anomaly', 'Normal']


In [8]:
!pip install opencv-python numpy scikit-learn xgboost


In [16]:
!pip install codecarbon


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 516.7/516.7 kB 18.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.4/66.4 kB 8.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 95.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.4/76.4 kB 9.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 92.5/92.5 kB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 5.4 MB/s eta 0:00:00
  Attempting uninstall: termcolor
    Found existing installation: termcolor 2.5.0
    Uninstalling termcolor-2.5.0:
      Successfully uninstalled termcolor-2.5.0
  Attempting uninstall: httpx
    Found existing installation: httpx 0.28.1
    Uninstalling httpx-0.28.1:
      Successfully uninstalled httpx-0.28.1


In [14]:
import os

image_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized/resized_256"  # 🔥 Modifie ici

# Vérifier si le dossier existe
if not os.path.exists(image_folder):
    raise ValueError(f"❌ Le dossier {image_folder} n'existe pas. Vérifie le chemin !")

# Lister les fichiers disponibles
for category in ["Anomaly", "Normal"]:
    category_path = os.path.join(image_folder, category)

    if not os.path.exists(category_path):
        print(f"⚠️ Dossier introuvable : {category_path}")
        continue

    print(f"\n📂 Contenu du dossier '{category}':")
    print(os.listdir(category_path)[:10])  # Affiche les 10 premiers fichiers



📂 Contenu du dossier 'Anomaly':
['016.JPG', '007.JPG', '004.JPG', '002.JPG', '006.JPG', '001.JPG', '000.JPG', '003.JPG', '005.JPG', '065.JPG']

📂 Contenu du dossier 'Normal':
['0029.JPG', '0028.JPG', '0022.JPG', '0021.JPG', '0026.JPG', '0024.JPG', '0025.JPG', '0023.JPG', '0027.JPG', '0008.JPG']


In [15]:
import os
import cv2
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report

# 📂 Dossier contenant les images redimensionnées
image_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized/resized_256"

# 📏 Taille des images
image_size = 256

# 📂 Vérification du dossier
if not os.path.exists(image_folder):
    raise ValueError(f"❌ Le dossier {image_folder} n'existe pas. Vérifie le chemin !")

# 📂 Chargement de toutes les images (sans filtre)
X = []
y = []

categories = ["Anomaly", "Normal"]
for label, category in enumerate(categories):
    category_path = os.path.join(image_folder, category)

    # Vérifier que le dossier existe
    if not os.path.exists(category_path):
        print(f"⚠️ Dossier introuvable : {category_path}")
        continue  # Passer cette catégorie si le dossier n'existe pas

    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            X.append(img.flatten())  # Convertir l’image en vecteur
            y.append(label)  # Attribuer la classe (0=Anomaly, 1=Normal)

# 📊 Vérifier si des images ont été chargées
if len(X) == 0:
    raise ValueError("❌ Aucune image trouvée ! Vérifie les fichiers et le dossier.")

X = np.array(X)
y = np.array(y)

# 📌 Tester avec différents pourcentages de données
percentages = [100, 80, 60, 40, 20]
results = {}

for p in percentages:
    print(f"\n🚀 Test avec {p}% des données...")

    # Sélectionner un sous-ensemble des données
    if p == 100:
        X_sample = X  # Prendre toutes les données
        y_sample = y
    else:
        X_sample, _, y_sample, _ = train_test_split(X, y, train_size=p/100, random_state=42)

    # Vérifier qu'il reste assez de données pour diviser en train/test
    if len(X_sample) < 2:
        print(f"⚠️ Pas assez de données pour {p}%, on passe au suivant.")
        continue  # Passer au prochain pourcentage

    # Diviser en train/test (80/20 sur cet échantillon)
    X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

    # 📌 Tester deux modèles : SVM et XGBoost
    models = {
        "SVM": SVC(kernel="linear"),
        "XGBoost": XGBClassifier(use_label_encoder=False, eval_metric="logloss")
    }

    for model_name, model in models.items():
        print(f"\n🔄 Entraînement du modèle {model_name} avec {p}% des données...")

        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"✅ {model_name} → Précision: {accuracy:.4f}, Temps d'entraînement: {training_time:.2f}s")

        results[f"{model_name}_{p}%"] = {
            "accuracy": accuracy,
            "training_time": training_time
        }

# 📊 Comparaison finale
print("\n📊 Comparaison des modèles et des pourcentages :")
for key, res in results.items():
    print(f"{key} → Précision: {res['accuracy']:.4f}, Temps d'entraînement: {res['training_time']:.2f}s")



🚀 Test avec 100% des données...

🔄 Entraînement du modèle SVM avec 100% des données...
✅ SVM → Précision: 0.9409, Temps d'entraînement: 11.13s

🔄 Entraînement du modèle XGBoost avec 100% des données...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:12:02] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ XGBoost → Précision: 0.9318, Temps d'entraînement: 62.56s

🚀 Test avec 80% des données...

🔄 Entraînement du modèle SVM avec 80% des données...
✅ SVM → Précision: 0.9659, Temps d'entraînement: 5.47s

🔄 Entraînement du modèle XGBoost avec 80% des données...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:13:11] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ XGBoost → Précision: 0.9602, Temps d'entraînement: 48.16s

🚀 Test avec 60% des données...

🔄 Entraînement du modèle SVM avec 60% des données...
✅ SVM → Précision: 0.9167, Temps d'entraînement: 2.39s

🔄 Entraînement du modèle XGBoost avec 60% des données...


...

✅ XGBoost → Précision: 0.9470, Temps d'entraînement: 46.96s

🚀 Test avec 40% des données...

🔄 Entraînement du modèle SVM avec 40% des données...
✅ SVM → Précision: 0.9205, Temps d'entraînement: 1.39s

🔄 Entraînement du modèle XGBoost avec 40% des données...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:14:46] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ XGBoost → Précision: 0.9545, Temps d'entraînement: 28.47s

🚀 Test avec 20% des données...

🔄 Entraînement du modèle SVM avec 20% des données...
✅ SVM → Précision: 0.9773, Temps d'entraînement: 0.44s

🔄 Entraînement du modèle XGBoost avec 20% des données...


/usr/local/lib/python3.11/dist-packages/xgboost/core.py:158: UserWarning: [10:15:14] WARNING: /workspace/src/learner.cc:740: 
Parameters: { "use_label_encoder" } are not used.

  warnings.warn(smsg, UserWarning)


✅ XGBoost → Précision: 0.9545, Temps d'entraînement: 10.40s

📊 Comparaison des modèles et des pourcentages :
SVM_100% → Précision: 0.9409, Temps d'entraînement: 11.13s
XGBoost_100% → Précision: 0.9318, Temps d'entraînement: 62.56s
SVM_80% → Précision: 0.9659, Temps d'entraînement: 5.47s
XGBoost_80% → Précision: 0.9602, Temps d'entraînement: 48.16s
SVM_60% → Précision: 0.9167, Temps d'entraînement: 2.39s
XGBoost_60% → Précision: 0.9470, Temps d'entraînement: 46.96s
SVM_40% → Précision: 0.9205, Temps d'entraînement: 1.39s
XGBoost_40% → Précision: 0.9545, Temps d'entraînement: 28.47s
SVM_20% → Précision: 0.9773, Temps d'entraînement: 0.44s
XGBoost_20% → Précision: 0.9545, Temps d'entraînement: 10.40s


In [17]:
import os
import cv2
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score, classification_report
from codecarbon import EmissionsTracker  # 🚀 Ajout du suivi carbone

# 📂 Dossier contenant les images redimensionnées
image_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized/resized_256"

# 📏 Taille des images
image_size = 256

# 📂 Vérification du dossier
if not os.path.exists(image_folder):
    raise ValueError(f"❌ Le dossier {image_folder} n'existe pas. Vérifie le chemin !")

# 📂 Chargement de toutes les images
X = []
y = []

categories = ["Anomaly", "Normal"]
for label, category in enumerate(categories):
    category_path = os.path.join(image_folder, category)

    if not os.path.exists(category_path):
        print(f"⚠️ Dossier introuvable : {category_path}")
        continue

    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            X.append(img.flatten())
            y.append(label)

# 📊 Vérifier si des images ont été chargées
if len(X) == 0:
    raise ValueError("❌ Aucune image trouvée ! Vérifie les fichiers et le dossier.")

X = np.array(X)
y = np.array(y)

# 📌 Tester avec différents pourcentages de données
percentages = [100, 80, 60, 40, 20]
results = {}

for p in percentages:
    print(f"\n🚀 Test avec {p}% des données...")

    # Sélectionner un sous-ensemble des données
    if p == 100:
        X_sample = X  # Prendre toutes les données
        y_sample = y
    else:
        X_sample, _, y_sample, _ = train_test_split(X, y, train_size=p/100, random_state=42)

    # Vérifier qu'il reste assez de données pour diviser en train/test
    if len(X_sample) < 2:
        print(f"⚠️ Pas assez de données pour {p}%, on passe au suivant.")
        continue

    # Diviser en train/test (80/20 sur cet échantillon)
    X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

    # 📌 Tester deux modèles : SVM et XGBoost
    models = {
        "SVM": SVC(kernel="linear"),
        "XGBoost": XGBClassifier(eval_metric="logloss")  # ⚠️ Suppression du paramètre inutile
    }

    for model_name, model in models.items():
        print(f"\n🔄 Entraînement du modèle {model_name} avec {p}% des données...")

        # 🔥 Démarrer le suivi de l'empreinte carbone
        tracker = EmissionsTracker(project_name=f"{model_name}_{p}%")
        tracker.start()

        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        # 🛑 Arrêter le suivi et récupérer les émissions de CO₂
        emissions = tracker.stop()

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"✅ {model_name} → Précision: {accuracy:.4f}, Temps d'entraînement: {training_time:.2f}s, Empreinte carbone: {emissions:.6f} kgCO₂")

        results[f"{model_name}_{p}%"] = {
            "accuracy": accuracy,
            "training_time": training_time,
            "carbon_emissions": emissions
        }

# 📊 Comparaison finale
print("\n📊 Comparaison des modèles et des empreintes carbones :")
for key, res in results.items():
    print(f"{key} → Précision: {res['accuracy']:.4f}, Temps: {res['training_time']:.2f}s, CO₂: {res['carbon_emissions']:.6f} kgCO₂")


[codecarbon INFO @ 10:19:26] [setup] RAM Tracking...
[codecarbon INFO @ 10:19:26] [setup] CPU Tracking...
[codecarbon WARNING @ 10:19:26] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU




🚀 Test avec 100% des données...

🔄 Entraînement du modèle SVM avec 100% des données...


[codecarbon WARNING @ 10:19:27] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:19:27] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:19:27] [setup] GPU Tracking...
[codecarbon INFO @ 10:19:27] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:19:27] >>> Tracker's metadata:
[codecarbon INFO @ 10:19:27]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:19:27]   Python version: 3.11.11
[codecarbon INFO @ 10:19:27]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:19:27]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:19:27]   CPU count: 2
[codecarbon INFO @ 10:19:27]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:19:27]   GPU count: 1
[codecarbon INFO @ 10:19:27]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:19:27] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:19:36] Energy consumed for RAM : 0

✅ SVM → Précision: 0.9409, Temps d'entraînement: 8.54s, Empreinte carbone: 0.000066 kgCO₂

🔄 Entraînement du modèle XGBoost avec 100% des données...


[codecarbon WARNING @ 10:19:38] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:19:38] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:19:38] [setup] GPU Tracking...
[codecarbon INFO @ 10:19:38] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:19:38] >>> Tracker's metadata:
[codecarbon INFO @ 10:19:38]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:19:38]   Python version: 3.11.11
[codecarbon INFO @ 10:19:38]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:19:38]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:19:38]   CPU count: 2
[codecarbon INFO @ 10:19:38]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:19:38]   GPU count: 1
[codecarbon INFO @ 10:19:38]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:19:39] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:19:54] Energy consumed for RAM : 0

...

✅ XGBoost → Précision: 0.9545, Temps d'entraînement: 39.38s, Empreinte carbone: 0.000310 kgCO₂

🚀 Test avec 20% des données...

🔄 Entraînement du modèle SVM avec 20% des données...


[codecarbon WARNING @ 10:24:23] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:24:23] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:24:23] [setup] GPU Tracking...
[codecarbon INFO @ 10:24:23] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:24:23] >>> Tracker's metadata:
[codecarbon INFO @ 10:24:23]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:24:23]   Python version: 3.11.11
[codecarbon INFO @ 10:24:23]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:24:23]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:24:23]   CPU count: 2
[codecarbon INFO @ 10:24:23]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:24:23]   GPU count: 1
[codecarbon INFO @ 10:24:23]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:24:23] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:24:24] Energy consumed for RAM : 0

✅ SVM → Précision: 0.9773, Temps d'entraînement: 0.47s, Empreinte carbone: 0.000004 kgCO₂

🔄 Entraînement du modèle XGBoost avec 20% des données...


[codecarbon WARNING @ 10:24:25] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:24:25] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:24:25] [setup] GPU Tracking...
[codecarbon INFO @ 10:24:25] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:24:25] >>> Tracker's metadata:
[codecarbon INFO @ 10:24:25]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:24:25]   Python version: 3.11.11
[codecarbon INFO @ 10:24:25]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:24:25]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:24:25]   CPU count: 2
[codecarbon INFO @ 10:24:25]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:24:25]   GPU count: 1
[codecarbon INFO @ 10:24:25]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:24:25] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:24:40] Energy consumed for RAM : 0

✅ XGBoost → Précision: 0.9545, Temps d'entraînement: 15.57s, Empreinte carbone: 0.000123 kgCO₂

📊 Comparaison des modèles et des empreintes carbones :
SVM_100% → Précision: 0.9409, Temps: 8.54s, CO₂: 0.000066 kgCO₂
XGBoost_100% → Précision: 0.9318, Temps: 98.27s, CO₂: 0.000763 kgCO₂
SVM_80% → Précision: 0.9659, Temps: 5.54s, CO₂: 0.000043 kgCO₂
XGBoost_80% → Précision: 0.9602, Temps: 75.39s, CO₂: 0.000590 kgCO₂
SVM_60% → Précision: 0.9167, Temps: 2.39s, CO₂: 0.000019 kgCO₂
XGBoost_60% → Précision: 0.9470, Temps: 48.55s, CO₂: 0.000382 kgCO₂
SVM_40% → Précision: 0.9205, Temps: 1.45s, CO₂: 0.000012 kgCO₂
XGBoost_40% → Précision: 0.9545, Temps: 39.38s, CO₂: 0.000310 kgCO₂
SVM_20% → Précision: 0.9773, Temps: 0.47s, CO₂: 0.000004 kgCO₂
XGBoost_20% → Précision: 0.9545, Temps: 15.57s, CO₂: 0.000123 kgCO₂


In [18]:
import os
import cv2
import numpy as np
import time
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
from codecarbon import EmissionsTracker

# 📂 Dossier contenant les images redimensionnées
image_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized/resized_256"

# 📏 Taille des images
image_size = 256

# 📂 Vérification du dossier
if not os.path.exists(image_folder):
    raise ValueError(f"❌ Le dossier {image_folder} n'existe pas. Vérifie le chemin !")

# 📂 Chargement de toutes les images
X = []
y = []

categories = ["Anomaly", "Normal"]
for label, category in enumerate(categories):
    category_path = os.path.join(image_folder, category)

    if not os.path.exists(category_path):
        print(f"⚠️ Dossier introuvable : {category_path}")
        continue

    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)

        if img is not None:
            img = cv2.resize(img, (image_size, image_size))
            X.append(img.flatten())  # Convertir en vecteur
            y.append(label)

# 📊 Vérifier si des images ont été chargées
if len(X) == 0:
    raise ValueError("❌ Aucune image trouvée ! Vérifie les fichiers et le dossier.")

X = np.array(X)
y = np.array(y)

# 📌 Tester avec différents pourcentages de données
percentages = [100, 80, 60, 40, 20]
results = {}

for p in percentages:
    print(f"\n🚀 Test avec {p}% des données...")

    # Sélectionner un sous-ensemble des données
    if p == 100:
        X_sample = X  # Prendre toutes les données
        y_sample = y
    else:
        X_sample, _, y_sample, _ = train_test_split(X, y, train_size=p/100, random_state=42)

    # Vérifier qu'il reste assez de données pour diviser en train/test
    if len(X_sample) < 2:
        print(f"⚠️ Pas assez de données pour {p}%, on passe au suivant.")
        continue

    # Diviser en train/test (80/20 sur cet échantillon)
    X_train, X_test, y_train, y_test = train_test_split(X_sample, y_sample, test_size=0.2, random_state=42)

    # 📌 Tester SVM et XGBoost optimisé
    models = {
        "SVM": SVC(kernel="linear"),
        "XGBoost": XGBClassifier(n_estimators=50, max_depth=3, eval_metric="logloss")  # ⚠️ Optimisation appliquée
    }

    for model_name, model in models.items():
        print(f"\n🔄 Entraînement du modèle {model_name} avec {p}% des données...")

        # 🔥 Démarrer le suivi de l'empreinte carbone
        tracker = EmissionsTracker(project_name=f"{model_name}_{p}%")
        tracker.start()

        start_time = time.time()
        model.fit(X_train, y_train)
        training_time = time.time() - start_time

        # 🛑 Arrêter le suivi et récupérer les émissions de CO₂
        emissions = tracker.stop()

        y_pred = model.predict(X_test)
        accuracy = accuracy_score(y_test, y_pred)

        print(f"✅ {model_name} → Précision: {accuracy:.4f}, Temps d'entraînement: {training_time:.2f}s, CO₂: {emissions:.6f} kgCO₂")

        results[f"{model_name}_{p}%"] = {
            "accuracy": accuracy,
            "training_time": training_time,
            "carbon_emissions": emissions
        }

# 📊 Comparaison finale
print("\n📊 Comparaison des modèles et des empreintes carbones :")
for key, res in results.items():
    print(f"{key} → Précision: {res['accuracy']:.4f}, Temps: {res['training_time']:.2f}s, CO₂: {res['carbon_emissions']:.6f} kgCO₂")


[codecarbon INFO @ 10:32:33] [setup] RAM Tracking...
[codecarbon INFO @ 10:32:33] [setup] CPU Tracking...
[codecarbon WARNING @ 10:32:33] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU




🚀 Test avec 100% des données...

🔄 Entraînement du modèle SVM avec 100% des données...


[codecarbon WARNING @ 10:32:34] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:32:34] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:32:34] [setup] GPU Tracking...
[codecarbon INFO @ 10:32:34] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:32:34] >>> Tracker's metadata:
[codecarbon INFO @ 10:32:34]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:32:34]   Python version: 3.11.11
[codecarbon INFO @ 10:32:34]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:32:34]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:32:34]   CPU count: 2
[codecarbon INFO @ 10:32:34]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:32:34]   GPU count: 1
[codecarbon INFO @ 10:32:34]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:32:35] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:32:43] Energy consumed for RAM : 0

✅ SVM → Précision: 0.9409, Temps d'entraînement: 8.24s, CO₂: 0.000065 kgCO₂

🔄 Entraînement du modèle XGBoost avec 100% des données...


[codecarbon WARNING @ 10:32:46] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:32:46] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:32:46] [setup] GPU Tracking...
[codecarbon INFO @ 10:32:46] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:32:46] >>> Tracker's metadata:
[codecarbon INFO @ 10:32:46]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:32:46]   Python version: 3.11.11
[codecarbon INFO @ 10:32:46]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:32:46]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:32:46]   CPU count: 2
[codecarbon INFO @ 10:32:46]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:32:46]   GPU count: 1
[codecarbon INFO @ 10:32:46]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:32:46] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:33:01] Energy consumed for RAM : 0

...

✅ XGBoost → Précision: 0.9432, Temps d'entraînement: 20.86s, CO₂: 0.000164 kgCO₂

🚀 Test avec 20% des données...

🔄 Entraînement du modèle SVM avec 20% des données...


[codecarbon WARNING @ 10:35:57] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:35:57] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:35:57] [setup] GPU Tracking...
[codecarbon INFO @ 10:35:57] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:35:57] >>> Tracker's metadata:
[codecarbon INFO @ 10:35:57]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:35:57]   Python version: 3.11.11
[codecarbon INFO @ 10:35:57]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:35:57]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:35:57]   CPU count: 2
[codecarbon INFO @ 10:35:57]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:35:57]   GPU count: 1
[codecarbon INFO @ 10:35:57]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:35:57] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:35:57] Energy consumed for RAM : 0

✅ SVM → Précision: 0.9773, Temps d'entraînement: 0.50s, CO₂: 0.000004 kgCO₂

🔄 Entraînement du modèle XGBoost avec 20% des données...


[codecarbon WARNING @ 10:35:59] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 10:35:59] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:35:59] [setup] GPU Tracking...
[codecarbon INFO @ 10:35:59] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 10:35:59] >>> Tracker's metadata:
[codecarbon INFO @ 10:35:59]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 10:35:59]   Python version: 3.11.11
[codecarbon INFO @ 10:35:59]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 10:35:59]   Available RAM : 12.675 GB
[codecarbon INFO @ 10:35:59]   CPU count: 2
[codecarbon INFO @ 10:35:59]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 10:35:59]   GPU count: 1
[codecarbon INFO @ 10:35:59]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 10:35:59] Saving emissions data to file /content/emissions.csv
[codecarbon INFO @ 10:36:09] Energy consumed for RAM : 0

✅ XGBoost → Précision: 0.9545, Temps d'entraînement: 10.25s, CO₂: 0.000080 kgCO₂

📊 Comparaison des modèles et des empreintes carbones :
SVM_100% → Précision: 0.9409, Temps: 8.24s, CO₂: 0.000065 kgCO₂
XGBoost_100% → Précision: 0.9364, Temps: 63.97s, CO₂: 0.000503 kgCO₂
SVM_80% → Précision: 0.9659, Temps: 9.44s, CO₂: 0.000075 kgCO₂
XGBoost_80% → Précision: 0.9489, Temps: 50.20s, CO₂: 0.000395 kgCO₂
SVM_60% → Précision: 0.9167, Temps: 4.75s, CO₂: 0.000038 kgCO₂
XGBoost_60% → Précision: 0.9470, Temps: 28.72s, CO₂: 0.000226 kgCO₂
SVM_40% → Précision: 0.9205, Temps: 1.44s, CO₂: 0.000011 kgCO₂
XGBoost_40% → Précision: 0.9432, Temps: 20.86s, CO₂: 0.000164 kgCO₂
SVM_20% → Précision: 0.9773, Temps: 0.50s, CO₂: 0.000004 kgCO₂
XGBoost_20% → Précision: 0.9545, Temps: 10.25s, CO₂: 0.000080 kgCO₂


In [24]:
!pip install ace-tools

In [29]:
import tensorflow as tf
import numpy as np
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from codecarbon import EmissionsTracker

# 📂 Dossier des images redimensionnées
image_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized/resized_256"

# 📏 Taille des images
image_size = (224, 224)  # 📌 Taille standard pour MobileNet et EfficientNet

# 📌 Chargement des images en numpy
X = []
y = []

categories = ["Anomaly", "Normal"]
for label, category in enumerate(categories):
    category_path = os.path.join(image_folder, category)

    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
        img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalisation
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

# 📌 Séparer les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Définir les modèles Deep Learning
def build_model(base_model):
    base_model.trainable = False  # Transfer Learning : on entraîne seulement la dernière couche
    inputs = tf.keras.Input(shape=image_size + (3,))
    x = base_model(inputs, training=False)
    x = GlobalAveragePooling2D()(x)
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])
    return model

models = {
    "MobileNetV2": build_model(MobileNetV2(weights="imagenet", include_top=False, input_shape=image_size + (3,))),
    "EfficientNetB0": build_model(EfficientNetB0(weights="imagenet", include_top=False, input_shape=image_size + (3,)))
}

# 📊 Tester les modèles et mesurer empreinte carbone
results = {}

for model_name, model in models.items():
    print(f"\n🔄 Entraînement du modèle {model_name}...")

    tracker = EmissionsTracker(project_name=model_name)
    tracker.start()

    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=5, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    training_time = time.time() - start_time

    emissions = tracker.stop()

    accuracy = history.history["val_accuracy"][-1]  # Dernière précision de validation
    results[model_name] = {"accuracy": accuracy, "training_time": training_time, "carbon_emissions": emissions}

# 📊 Afficher les résultats sans ace_tools
import pandas as pd
from IPython.display import display

# Convertir les résultats en DataFrame
df_dl = pd.DataFrame(results).T

# 📊 Affichage correct du tableau
display(df_dl)



[codecarbon INFO @ 11:09:39] [setup] RAM Tracking...
[codecarbon INFO @ 11:09:39] [setup] CPU Tracking...
[codecarbon WARNING @ 11:09:39] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU




🔄 Entraînement du modèle MobileNetV2...


[codecarbon WARNING @ 11:09:40] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 11:09:40] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 11:09:40] [setup] GPU Tracking...
[codecarbon INFO @ 11:09:40] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:09:40] >>> Tracker's metadata:
[codecarbon INFO @ 11:09:40]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 11:09:40]   Python version: 3.11.11
[codecarbon INFO @ 11:09:40]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 11:09:40]   Available RAM : 12.675 GB
[codecarbon INFO @ 11:09:40]   CPU count: 2
[codecarbon INFO @ 11:09:40]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 11:09:40]   GPU count: 1
[codecarbon INFO @ 11:09:40]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 11:09:40] Saving emissions data to file /content/emissions.csv


Epoch 1/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 0s 100ms/step - accuracy: 0.9164 - loss: 0.2879

[codecarbon INFO @ 11:09:55] Energy consumed for RAM : 0.000020 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 11:09:55] Energy consumed for all CPUs : 0.000177 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:09:55] Energy consumed for all GPUs : 0.000151 kWh. Total GPU Power : 36.20752328898104 W
[codecarbon INFO @ 11:09:55] 0.000348 kWh of electricity used since the beginning.


...

28/28 ━━━━━━━━━━━━━━━━━━━━ 39s 771ms/step - accuracy: 0.9102 - loss: 0.3878 - val_accuracy: 0.8909 - val_loss: 0.3722
Epoch 2/5


[codecarbon INFO @ 11:10:57] Energy consumed for RAM : 0.000059 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 11:10:57] Energy consumed for all CPUs : 0.000531 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:10:57] Energy consumed for all GPUs : 0.000434 kWh. Total GPU Power : 33.747302465049046 W
[codecarbon INFO @ 11:10:57] 0.001024 kWh of electricity used since the beginning.


28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 52ms/step - accuracy: 0.9195 - loss: 0.2830 - val_accuracy: 0.8909 - val_loss: 0.3454
Epoch 3/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 48ms/step - accuracy: 0.9065 - loss: 0.3125 - val_accuracy: 0.8909 - val_loss: 0.3506
Epoch 4/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 1s 51ms/step - accuracy: 0.9266 - loss: 0.2617 - val_accuracy: 0.8909 - val_loss: 0.3448
Epoch 5/5
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 49ms/step - accuracy: 0.8997 - loss: 0.3248 - val_accuracy: 0.8909 - val_loss: 0.3548


[codecarbon INFO @ 11:11:10] Energy consumed for RAM : 0.000077 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 11:11:10] Energy consumed for all CPUs : 0.000686 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:11:10] Energy consumed for all GPUs : 0.000608 kWh. Total GPU Power : 47.785635981601445 W
[codecarbon INFO @ 11:11:10] 0.001371 kWh of electricity used since the beginning.


,accuracy,training_time,carbon_emissions
MobileNetV2,0.909091,30.891210,0.000260
EfficientNetB0,0.890909,58.169224,0.000479


In [30]:
import tensorflow as tf
import numpy as np
import os
import time
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import MobileNetV2, EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from codecarbon import EmissionsTracker

# 📂 Dossier des images redimensionnées
image_folder = "/content/drive/MyDrive/HAK/candle/Data/Images_Resized/resized_256"

# 📏 Nouvelle taille des images (optimisation : 160x160 pour accélérer)
image_size = (160, 160)

# 📌 Chargement des images
X = []
y = []

categories = ["Anomaly", "Normal"]
for label, category in enumerate(categories):
    category_path = os.path.join(image_folder, category)

    for img_name in os.listdir(category_path):
        img_path = os.path.join(category_path, img_name)
        img = tf.keras.preprocessing.image.load_img(img_path, target_size=image_size)
        img = tf.keras.preprocessing.image.img_to_array(img) / 255.0  # Normalisation
        X.append(img)
        y.append(label)

X = np.array(X)
y = np.array(y)

# 📌 Séparer les données en train/test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# 📌 Définir les modèles optimisés (fine-tuning activé)
def build_optimized_model(base_model):
    base_model.trainable = True  # Débloquer les dernières couches pour le fine-tuning
    inputs = tf.keras.Input(shape=image_size + (3,))
    x = base_model(inputs, training=True)
    x = GlobalAveragePooling2D()(x)
    x = Dropout(0.3)(x)  # Ajout d'un Dropout pour éviter le surajustement
    outputs = Dense(1, activation="sigmoid")(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.0001),
                  loss="binary_crossentropy", metrics=["accuracy"])
    return model

# 📌 Créer les modèles optimisés
models = {
    "MobileNetV2": build_optimized_model(MobileNetV2(weights="imagenet", include_top=False, input_shape=image_size + (3,))),
    "EfficientNetB0": build_optimized_model(EfficientNetB0(weights="imagenet", include_top=False, input_shape=image_size + (3,)))
}

# 📊 Tester les modèles optimisés et mesurer empreinte carbone
results = {}

for model_name, model in models.items():
    print(f"\n🔄 Entraînement du modèle {model_name} (optimisé)...")

    tracker = EmissionsTracker(project_name=f"{model_name}_Optimized")
    tracker.start()

    start_time = time.time()
    history = model.fit(X_train, y_train, epochs=15, batch_size=32, validation_data=(X_test, y_test), verbose=1)
    training_time = time.time() - start_time

    emissions = tracker.stop()

    accuracy = history.history["val_accuracy"][-1]  # Dernière précision de validation
    results[model_name] = {"accuracy": accuracy, "training_time": training_time, "carbon_emissions": emissions}

# 📊 Afficher les résultats
import pandas as pd
from IPython.display import display
df_dl_optimized = pd.DataFrame(results).T
display(df_dl_optimized)


9406464/9406464 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


[codecarbon INFO @ 11:17:21] [setup] RAM Tracking...
[codecarbon INFO @ 11:17:21] [setup] CPU Tracking...
[codecarbon WARNING @ 11:17:21] No CPU tracking mode found. Falling back on CPU constant mode. 
 Linux OS detected: Please ensure RAPL files exist at \sys\class\powercap\intel-rapl to measure CPU




🔄 Entraînement du modèle MobileNetV2 (optimisé)...


[codecarbon WARNING @ 11:17:22] We saw that you have a Intel(R) Xeon(R) CPU @ 2.00GHz but we don't know it. Please contact us.
[codecarbon INFO @ 11:17:22] CPU Model on constant consumption mode: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 11:17:22] [setup] GPU Tracking...
[codecarbon INFO @ 11:17:22] Tracking Nvidia GPU via pynvml
[codecarbon INFO @ 11:17:22] >>> Tracker's metadata:
[codecarbon INFO @ 11:17:22]   Platform system: Linux-6.1.85+-x86_64-with-glibc2.35
[codecarbon INFO @ 11:17:22]   Python version: 3.11.11
[codecarbon INFO @ 11:17:22]   CodeCarbon version: 2.8.3
[codecarbon INFO @ 11:17:22]   Available RAM : 12.675 GB
[codecarbon INFO @ 11:17:22]   CPU count: 2
[codecarbon INFO @ 11:17:22]   CPU model: Intel(R) Xeon(R) CPU @ 2.00GHz
[codecarbon INFO @ 11:17:22]   GPU count: 1
[codecarbon INFO @ 11:17:22]   GPU model: 1 x Tesla T4
[codecarbon INFO @ 11:17:22] Saving emissions data to file /content/emissions.csv


Epoch 1/15


...

28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 84ms/step - accuracy: 0.9892 - loss: 0.0389 - val_accuracy: 0.8909 - val_loss: 0.6239
Epoch 7/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 82ms/step - accuracy: 0.9936 - loss: 0.0265 - val_accuracy: 0.8909 - val_loss: 0.5197
Epoch 8/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.9891 - loss: 0.0420 - val_accuracy: 0.8909 - val_loss: 0.5620
Epoch 9/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 92ms/step - accuracy: 0.9951 - loss: 0.0197 - val_accuracy: 0.8909 - val_loss: 0.5003
Epoch 10/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 5s 87ms/step - accuracy: 0.9836 - loss: 0.0464 - val_accuracy: 0.8909 - val_loss: 0.6225
Epoch 11/15
27/28 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - accuracy: 0.9874 - loss: 0.0273

[codecarbon INFO @ 11:23:14] Energy consumed for RAM : 0.000277 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 11:23:14] Energy consumed for all CPUs : 0.002478 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:23:14] Energy consumed for all GPUs : 0.002268 kWh. Total GPU Power : 62.88691863391341 W
[codecarbon INFO @ 11:23:14] 0.005024 kWh of electricity used since the beginning.


28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 80ms/step - accuracy: 0.9876 - loss: 0.0272 - val_accuracy: 0.8182 - val_loss: 0.6619
Epoch 12/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 82ms/step - accuracy: 0.9916 - loss: 0.0445 - val_accuracy: 0.8727 - val_loss: 0.6207
Epoch 13/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 87ms/step - accuracy: 0.9904 - loss: 0.0309 - val_accuracy: 0.8727 - val_loss: 0.6111
Epoch 14/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 2s 83ms/step - accuracy: 0.9948 - loss: 0.0209 - val_accuracy: 0.1409 - val_loss: 0.7597
Epoch 15/15
28/28 ━━━━━━━━━━━━━━━━━━━━ 3s 86ms/step - accuracy: 0.9977 - loss: 0.0082 - val_accuracy: 0.1091 - val_loss: 0.9562


[codecarbon INFO @ 11:23:24] Energy consumed for RAM : 0.000291 kWh. RAM Power : 4.7530388832092285 W
[codecarbon INFO @ 11:23:24] Energy consumed for all CPUs : 0.002600 kWh. Total CPU Power : 42.5 W
[codecarbon INFO @ 11:23:24] Energy consumed for all GPUs : 0.002456 kWh. Total GPU Power : 65.80404640495269 W
[codecarbon INFO @ 11:23:24] 0.005347 kWh of electricity used since the beginning.


,accuracy,training_time,carbon_emissions
MobileNetV2,0.890909,139.683779,0.001207
EfficientNetB0,0.109091,220.381721,0.001867
